Instalações

In [2]:
from sklearn.preprocessing import OneHotEncoder, normalize
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
import csv
from sklearn.metrics.pairwise import pairwise_distances_argmin_min
import requests

/tmp/ipykernel_9115/4134354371.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Essa função é usada para a gambiarra de baixar as imagens

In [15]:
def baixar_imagem(url, nome_arquivo, id):
    try:
        # Faz uma requisição GET para a URL da imagem
        resposta = requests.get(url)

        # Verifica se a requisição foi bem-sucedida
        if resposta.status_code == 200:
            # Abre o arquivo no modo de escrita binária ('wb') e escreve o conteúdo da resposta
            with open(f'{nome_arquivo}{id}.jpg', 'wb') as arquivo:
                arquivo.write(resposta.content)
        else:
            print("Falha ao baixar a imagem:", resposta.status_code)
    except Exception as e:
        print("Erro ao baixar a imagem:", str(e))

# Exemplo de uso
url = "http://assets.myntassets.com/v1/images/style/properties/3c7a1a7a39a32e91474d905ca3090bf1_images.jpg"
nome_arquivo = "C:/Users/eliab/Documents/projetao/"
baixar_imagem(url, nome_arquivo, 1)

Erro ao baixar a imagem: [Errno 2] No such file or directory: 'C:/Users/eliab/Documents/projetao/1.jpg'


### FUNÇÃO MUITO IMPORTANTE
Image_to_pixels -> Função que redimensiona e converte as imagens em um vetor de pixels.
Esta função é chamada no momento de estruturar os dados, na variável X(onde estarão todos os nossos dados) que aparece duas celulas a baixo.
Paraca imagem que será usada no treinamento, esta função será chamada uma vez, e recebe como parâmetro a própria imagem.

In [3]:
def image_to_pixels(image):


    # Redimensionar a imagem
    image = cv2.resize(image, (180, 180))

    # Converter para um array numpy
    pixels = image.flatten()
    pixels_array = np.array(pixels)

    #normalizando os pixels para um intervalo (-1,1)
    normalized_pixels = normalize([pixels_array], norm='l2', axis=1)

    return normalized_pixels[0]

Essa função é usada para a gambiarra de baixar as imagens

In [17]:
def separar_ids_links(arquivo_csv):
    ids = []
    links = []
    with open(arquivo_csv, 'r', newline='', encoding='utf-8') as csv_file:
        leitor_csv = csv.reader(csv_file)
        cont = 0
        for linha in leitor_csv:
            #ids.append(linha[0])
            if cont < 4000:
              cont +=1
              baixar_imagem(linha[1], "Roupas_images/", linha[0][:-4])
            else:
              break
            print(cont)


# Exemplo de uso
arquivo_csv = 'images.csv'
separar_ids_links(arquivo_csv)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277



### JUNTANDO AS IMAGENS NO FORMATO IDEAL PARA O TREINAMENTO
Pegando as imagens, redimensionando e convertendo em um vetor de pixels normalizado(-1,1)


In [4]:
X = []

# Diretório que contém as imagens
caminho_relativo ="Roupas_images"
diretorio = caminho_relativo  # Substitua pelo caminho do seu diretório

# Iterar sobre os arquivos no diretório
ids = [] #id[0] = id da imagem 0 em X
cont =0 
for nome_arquivo in os.listdir(diretorio):
    caminho_imagem = os.path.join(diretorio, nome_arquivo)
    if os.path.isfile(caminho_imagem) and cont < 1401:  # Verificar se o caminho é um arquivo
        imagem = cv2.imread(caminho_imagem)
        cont += 1
        X.append(image_to_pixels(imagem))
        ids.append(nome_arquivo[:-4])
X = np.array(X)
print(X)
ids = sorted(ids)
print(ids)

Premature end of JPEG file


[[0.00361198 0.00361198 0.00361198 ... 0.00361198 0.00361198 0.00361198]
 [0.0037947  0.0037947  0.0037947  ... 0.0037947  0.0037947  0.0037947 ]
 [0.0036772  0.0036772  0.0036772  ... 0.0036772  0.0036772  0.0036772 ]
 ...
 [0.0035625  0.0035625  0.0035625  ... 0.0035625  0.0035625  0.0035625 ]
 [0.00366483 0.00366483 0.00366483 ... 0.00366483 0.00366483 0.00366483]
 [0.0039757  0.0039757  0.0039757  ... 0.0039757  0.0039757  0.0039757 ]]
['10000', '10053', '10054', '10062', '10063', '10065', '10096', '10099', '10203', '10258', '10259', '10293', '10295', '10401', '10430', '10431', '10463', '10490', '10499', '10604', '10605', '10660', '10669', '10803', '10835', '10857', '11119', '11128', '11144', '11145', '11172', '11174', '11175', '11188', '11346', '11371', '11383', '11510', '11511', '11516', '11521', '11529', '11545', '11580', '11714', '11725', '11741', '11770', '11777', '11784', '11946', '11947', '11948', '11949', '11971', '12101', '12108', '12137', '12153', '12164', '12196', '12197

### ALINHANDO OS IDS DAS IMAGES.CSV COM OS IDS DE STYLES.CSV PARA QUE CADA DESCRIÇÃO CORRESPONDA A SUA RESPECTIVA IMAGEM NA HORA DE TREINAR

Copiando as labels e criando um novo dataframe que guar os rotulos dos dados

In [5]:
y_labels = []
arquivo_csv = "styles.csv"
with open(arquivo_csv, 'r', newline='', encoding='utf-8') as csv_file:
        leitor_csv = csv.reader(csv_file)
        for linha in leitor_csv:
            linha_id = linha[0]
            for id in ids:
                if linha_id == id:
                    y_labels.append(linha)

#ordenando as labels para combinarem com suas respectivas imagens
y_labels = sorted(y_labels, key=lambda x: x[0])
print(y_labels)


[['10000', 'Women', 'Apparel', 'Bottomwear', 'Skirts', 'White', 'Summer', '2011', 'Casual'], ['10053', 'Women', 'Apparel', 'Topwear', 'Tshirts', 'Pink', 'Summer', '2011', 'Casual'], ['10054', 'Boys', 'Apparel', 'Topwear', 'Tshirts', 'Blue', 'Summer', '2011', 'Sports'], ['10062', 'Men', 'Apparel', 'Topwear', 'Tshirts', 'Grey', 'Fall', '2011', 'Casual'], ['10063', 'Men', 'Apparel', 'Topwear', 'Tshirts', 'Yellow', 'Fall', '2011', 'Casual'], ['10065', 'Men', 'Apparel', 'Topwear', 'Tshirts', 'Yellow', 'Fall', '2011', 'Casual'], ['10096', 'Women', 'Footwear', 'Shoes', 'Sports Shoes', 'Blue', 'Fall', '2011', 'Sports'], ['10099', 'Men', 'Accessories', 'Watches', 'Watches', 'Grey', 'Winter', '2016', 'Casual'], ['10203', 'Women', 'Accessories', 'Bags', 'Handbags', 'Red', 'Winter', '2015', 'Casual'], ['10258', 'Men', 'Apparel', 'Bottomwear', 'Trousers', 'Navy Blue', 'Fall', '2011', 'Formal'], ['10259', 'Men', 'Apparel', 'Bottomwear', 'Trousers', 'Black', 'Fall', '2011', 'Formal'], ['10293', 'Men'

### TRANFORMANDO OS LABELS EM COLUNAS CODIFICADAS
Esta célula converte os dados categóricos para um formato que o modelo pode compreender.
É nesta etapa que definimos qual coluna será o rótulo que queremos prever.

In [6]:
# Definindo os nomes das colunas
colunas = ["id", "gender", "masterCategory", "subCategory", "articleType", "baseColour", "season", "year", "usage"]

# Criando o DataFrame
df = pd.DataFrame(y_labels, columns=colunas)

# Selecionar apenas as colunas categóricas
dados_categoricos = df.drop(columns=["id", "gender", "subCategory", "masterCategory", "articleType", "baseColour", "season", "year", "usage"])


#COLUNA QUE VAI SER PREVISTA
coluna_codificada = colunas[1]


# Ajustar e transformar os dados categóricos
# Criar um codificador one-hot
encoder = LabelEncoder()
dados_categoricos[coluna_codificada] = encoder.fit_transform(df[coluna_codificada])
#print(dados_categoricos)




### Separando dados de treinamento e iniciando o modelo

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, dados_categoricos, test_size=0.2, random_state=42)
modelo = KNeighborsClassifier(n_neighbors=5, weights='distance')
modelo.fit(X_train, y_train)


/home/codespace/.local/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(weights='distance')

### Testes -> ainda iniciando

In [8]:
y_pred = modelo.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do KNN com pesos personalizados:", accuracy)

Acurácia do KNN com pesos personalizados: 0.42704626334519574


In [10]:
img = cv2.imread("/workspaces/cabid/1.jpg")
img_processed = image_to_pixels(img)
predicted_label = modelo.predict([img_processed])
predicted_label = encoder.inverse_transform(predicted_label)
print(predicted_label)

['Women']


### Esta celula foi utilizada para apagar a última coluna.
Foi preciso apagar por que cada valor era uma frase e em alguns casos(quando havia uma virgula na frase) esbarrava em um erro ao tentar abrir o csv com pandas
Em segundo lugar aquele tipo de dados seria dificilmente bem processado pelo cluster

In [ ]:
novas_imagens = []
# Diretório que contém as imagens

# Iterar sobre os arquivos no diretório
imagem = cv2.imread("/content/test2.jpg")
imagem2 = cv2.imread("/content/teste1.jpg")


novas_imagens.append(image_to_pixels(imagem))
print(modelo.components_[1])
print(novas_imagens[0])
# Encontrar o cluster mais próximo da nova imagem
cluster_centroide_indices = pairwise_distances_argmin_min(novas_imagens, modelo.components_)[0]
cluster_centroide_indice = cluster_centroide_indices[0]

# Encontrar imagens semelhantes no cluster mais próximo
imagens_similares_indices = np.where(modelo.labels_ == cluster_centroide_indice)[0]

print("imgs ", imagens_similares_indices)